In [13]:
from vivarium_public_health.interactive import initialize_simulation_from_model_specification

sim = initialize_simulation_from_model_specification('/home/kate/code/vivarium/neonatal/neonatal/model_specs/test_lbwsg.yaml')
sim.configuration.update({'population': {'age_start': 0, 'age_end': 0.02, 'population_size': 1_000_000}})
sim.setup()

/home/kate/miniconda3/envs/neonatal/lib/python3.6/site-packages/vivarium/framework/configuration.py:35: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml.load(file_path)
/home/kate/miniconda3/envs/neonatal/lib/python3.6/site-packages/vivarium/config_tree.py:394: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data_dict = yaml.load(data_string)
/home/kate/miniconda3/envs/neonatal/lib/python3.6/site-packages/vivarium_public_health/population/data_transformations.py:156: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  pop_data = pd.concat([lower_bi

In [14]:
lbwsg = sim.get_value('low_birth_weight_and_short_gestation.exposure')

pop_start = sim.get_population()
import time
s = time.time()
at_start = lbwsg(pop_start.index)
e = time.time()
print(f'Calling lbwsg exp pipeline took {e - s} seconds.')

Calling lbwsg exp pipeline took 2.6035218238830566 seconds.


In [ ]:
at_start.value_counts()

In [ ]:
pop_start.age.hist()

In [15]:
from vivarium_public_health.dataset_manager import Artifact
art = Artifact(sim.configuration.input_data.artifact_path, 
               filter_terms=[f'draw == {sim.configuration.input_data.input_draw_number}'])
exp = art.load('risk_factor.low_birth_weight_and_short_gestation.exposure')

In [16]:
age_group_start = 0
age_group_end = 7/365
sex = 'Female'

pop_data = pop_start[(pop_start.age < age_group_end) & (pop_start.age >= age_group_start) 
                     & (pop_start.sex == sex)]
art_data = exp[(exp.age_group_start == age_group_start) & (exp.year_start == sim.clock.time.year) \
               & (exp.sex == sex)]

comp = art_data[['parameter', 'value']].set_index('parameter').rename(columns={'value': 'art'})
comp = comp.join((lbwsg(pop_data.index).value_counts()/len(pop_data.index)).rename('sim'))
comp['diff'] = (comp.art - comp.sim) / comp.art
comp

,art,sim,diff
parameter,,,
cat10,0.001619,0.001696,-0.047396
cat106,0.001242,0.001275,-0.026479
cat11,0.001187,0.001265,-0.065562
cat116,0.002362,0.002248,0.048077
cat117,0.009180,0.008856,0.035333
cat123,0.002918,0.002916,0.000451
cat14,0.001502,0.001446,0.037432
cat15,0.001652,0.001630,0.012937
cat17,0.003204,0.003177,0.008418


In [17]:
pop_data.shape

(475952, 9)

In [ ]:
#sim.run()